In [1]:
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, recall_score, balanced_accuracy_score, precision_score, make_scorer
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, train_test_split
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [26]:
params = {'n_estimators': [300, 400, 500, 600, 700],
              'learning_rate': [0.01, 0.02, 0.03, 0.05, 0.07],
              'gamma': [0.5, 1, 1.5, 2, 5],
              'max_depth': [3, 4, 5, 6],
              'subsample': [0.6, 0.8, 1.0],
              'colsample_bytree': [0.6, 0.8, 1.0],
              'min_child_weight': [1, 2, 3, 4, 5]}

seed = 42
st_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

def calc_scores(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    recall_0, recall_1 = recall_score(y_test, y_pred, pos_label=0), recall_score(y_test, y_pred, pos_label=1)
    precision_0, precision_1 =  precision_score(y_test, y_pred, pos_label=0), precision_score(y_test, y_pred, pos_label=1)
    acc = balanced_accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])
    arr = np.array([[acc, precision_0, recall_0, precision_1, recall_1,auc_score]])
    return pd.DataFrame(data=arr, columns=["balanced_accuracy", "recall_0", "precision_0", "recall_1", "precision_1", "auc"])

def recall_0(y_true, y_pred):
    return recall_score(y_true, y_pred, pos_label=0)

def precision_0(y_true, y_pred):
    return precision_score(y_true, y_pred, pos_label=0)

scoring = {"balanced_accuracy": make_scorer(balanced_accuracy_score),
           "recall_0": make_scorer(recall_0), "precision_0": make_scorer(precision_0),
           "recall_1": make_scorer(recall_score), "precision_1": make_scorer(precision_score), "auc": "roc_auc" }

#cross_validation

def print_score_comparison(raw_score, emb_score, target_feature="posOutcome",
                           header_1="Raw Score", header_2="Embedding Score"):
    print("\t\t{0}\n\t\t\t{1}\t\t{2}".format(target_feature, header_1, header_2))
    print("\t\t-----------------------------------------------")
    print("balanced_accuracy:\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["balanced_accuracy"].mean(), emb_score["balanced_accuracy"].mean()))
    print("precision_0:\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["precision_0"].mean(), emb_score["precision_0"].mean()))
    print("recall_0:\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["recall_0"].mean(), emb_score["recall_0"].mean()))
    print("precision_1:\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["precision_1"].mean(), emb_score["precision_1"].mean()))
    print("recall_1:\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["recall_1"].mean(), emb_score["recall_1"].mean()))
    print("auc:\t\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["auc"].mean(), emb_score["auc"].mean()))

def print_score_comparison_cv(raw_score, emb_score, target_feature="posOutcome",header_1="Raw Score", header_2="Embedding Score"):
    print("\t\t{0}\n\t\t\t{1}\t\t{2}".format(target_feature, header_1, header_2))
    print("\t\t-----------------------------------------------")
    print("balanced_accuracy:\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["test_balanced_accuracy"].mean(), emb_score["test_balanced_accuracy"].mean()))
    print("precision_0:\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["test_precision_0"].mean(), emb_score["test_precision_0"].mean()))
    print("recall_0:\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["test_recall_0"].mean(), emb_score["test_recall_0"].mean()))
    print("precision_1:\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["test_precision_1"].mean(), emb_score["test_precision_1"].mean()))
    print("recall_1:\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["test_recall_1"].mean(), emb_score["test_recall_1"].mean()))
    print("auc:\t\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["test_auc"].mean(), emb_score["test_auc"].mean()))

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time

    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

def param_tuning(X, y, n_folds=5, param_comb=25, scoring='roc_auc', jobs=12):
    xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    rand_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring=scoring, n_jobs=jobs,
                                   cv=skf.split(X, y), verbose=3, random_state=42)

    start_time = timer(None) # timing starts from this point for "start_time" variable
    rand_search.fit(X, y)
    timer(start_time)
    print("Best Score: {:.3%}".format(rand_search.best_score_))
    print(rand_search.best_params_)
    return rand_search

score_cols = ["test_balanced_accuracy","test_precision_0", "test_recall_0",
               "test_precision_1","test_recall_1", "test_auc"]

def get_scores(cv_results, score_keys=None, df_cols=None):
    if score_keys is None:
        score_keys = score_cols
    if df_cols is None:
        df_cols = score_cols
    scores = np.empty([1, len(score_keys)])
    for i, s in enumerate(score_keys):
        scores[0][i] = np.mean(cv_results[s])
    scores_df = pd.DataFrame(data=scores, columns=df_cols)
    return scores_df

def evaluate_embedding(path, train_df, test_df, target="posOutcome", params=None
                       ,n_jobs=-1):
    emb_df = pd.read_csv(path, sep="\t", index_col="patient_ID")
    X_train_emb, y_train_emb  = emb_df.loc[train_df.index,:], train_df.loc[:,target]
    X_test_emb, y_test_emb = emb_df.loc[test_df.index,:], test_df.loc[:,target]
    if params is None:
        rand_search_emb = param_tuning(X_train_emb, y_train_emb, jobs=n_jobs)
        clf_params = rand_search_emb.best_params_
    else:
        clf_params = params
    clf_emb = XGBClassifier(**clf_params)
    cv_res = cross_validate(clf_emb, X_train_emb, y_train_emb, scoring=scoring, n_jobs=n_jobs, verbose=1, return_train_score=True, cv=st_cv)
    cv_res_df = get_scores(cv_res)
    clf_emb.fit(X_train_emb, y_train_emb)
    test_scores_df = calc_scores(clf_emb, X_test_emb, y_test_emb)
    if params is None:
        return params, clf_emb, cv_res_df, test_scores_df
    else:
        return clf_emb, cv_res_df, test_scores_df

In [19]:
pos_outcome_df = pd.read_csv("datasets/combat15outcomes_latest.csv", index_col="patient_ID")
pos_outcome_df = pos_outcome_df.dropna(axis=0, subset=["posOutcome"])
pos_outcome_df = pos_outcome_df["posOutcome"]
train_df = pd.read_csv("datasets/train.csv", index_col="patient_ID")
test_df = pd.read_csv("datasets/test.csv", index_col="patient_ID")

In [7]:
params_moses_500 = {'subsample': 1.0,
 'n_estimators': 400,
 'min_child_weight': 3,
 'max_depth': 5,
 'learning_rate': 0.01,
 'gamma': 1,
 'colsample_bytree': 0.6}


In [20]:
clf_moses500, cv_scores_moses500, test_scores_moses500 = evaluate_embedding("datasets/embedding-vectors/genexpr_clinicaldata/property_vector_moses500_wopln_2021-01-28.csv", train_df, test_df,
    params=params_moses_500)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.4min finished


[11:11:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [21]:
cv_scores_moses500.mean()

test_balanced_accuracy    0.752695
test_precision_0          0.767297
test_recall_0             0.764848
test_precision_1          0.739075
test_recall_1             0.740541
test_auc                  0.816806
dtype: float64

In [22]:
test_scores_moses500.mean()

balanced_accuracy    0.764844
recall_0             0.783237
precision_0          0.765537
recall_1             0.745399
precision_1          0.764151
auc                  0.821510
dtype: float64

In [27]:
mrmr50_params = {'subsample': 0.8,
 'n_estimators': 400,
 'min_child_weight': 3,
 'max_depth': 5,
 'learning_rate': 0.03,
 'gamma': 1.5,
 'colsample_bytree': 0.8}

clf_mrmr50, cv_scores_mrmr50_df, test_scores_mrmr50_df = evaluate_embedding("datasets/embedding-vectors/genexpr_only/property_vector_mrmr_ft50_2021-01-29.csv", train_df, test_df, params=mrmr50_params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.0min finished


[18:12:19] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [29]:
print("CV Score: \n{0}\n".format(cv_scores_mrmr50_df.mean()))
print("Test Score:\n{0}\n".format(test_scores_mrmr50_df.mean()))


CV Score: 
test_balanced_accuracy    0.741585
test_precision_0          0.745276
test_recall_0             0.781818
test_precision_1          0.742950
test_recall_1             0.701351
test_auc                  0.795192
dtype: float64

Test Score:
balanced_accuracy    0.725696
recall_0             0.731707
precision_0          0.762712
recall_1             0.722772
precision_1          0.688679
auc                  0.777396
dtype: float64



In [30]:
mrmr50_cl_params, clf_mrmr50_cl, cv_mrmr50_cl, test_scores_mrmr50_cl_df = evaluate_embedding("datasets/embedding-vectors/genexpr_clinicaldata/property_vector_mrmr_ft50_wopln_2021-01-29.csv", train_df, test_df)
print(mrmr50_cl_params)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[18:40:28] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:40:28] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 26 minutes and 32.78 seconds.
Best Score: 80.599%
{'subsample': 1.0, 'n_estimators': 600, 'min_child_weight': 2, 'max_depth': 6, 'learning_rate': 0.02, 'gamma': 2, 'colsample_bytree': 0.6}
[18:47:27] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was ch

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed: 23.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.8min finished


ValueError: too many values to unpack (expected 3)

In [32]:
print("CV Score: \n{0}\n".format(cv_mrmr50_cl.mean()))
print("Test Score:\n{0}\n".format(test_scores_mrmr50_cl_df.mean()))

NameError: name 'cv_mrmr50_cl' is not defined

In [ ]:
print_score_comparison(test_scores_mrmr50_df, test_scores_mrmr50_cl_df,
                       header_1="MRMR 50 Emb w/o CL",
                       header_2="MRMR 50 Emb w CL")


In [33]:
mrmr50_pln_params, clf_mrmr50_pln, cv_mrmr50_pln_df, test_scores_mrmr50_pln_df = evaluate_embedding("datasets/embedding-vectors/genexpr_clinicaldata_allgo-pw/property_vector_mrmr_ft50_all_2021-02-05.csv", train_df, test_df)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[19:25:43] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:25:43] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 26 minutes and 18.88 seconds.
Best Score: 79.109%
{'subsample': 0.8, 'n_estimators': 600, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.01, 'gamma': 5, 'colsample_bytree': 0.6}
[19:32:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was ch

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed: 23.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.6min finished


In [34]:
print("CV Score: \n{0}\n".format(cv_mrmr50_pln_df.mean()))
print("Test Score:\n{0}\n".format(test_scores_mrmr50_pln_df.mean()))

CV Score: 
test_balanced_accuracy    0.739816
test_precision_0          0.740782
test_recall_0             0.789091
test_precision_1          0.745805
test_recall_1             0.690541
test_auc                  0.791089
dtype: float64

Test Score:
balanced_accuracy    0.727561
recall_0             0.725849
precision_0          0.785311
recall_1             0.737024
precision_1          0.669811
auc                  0.782619
dtype: float64



In [35]:
print_score_comparison(test_scores_mrmr50_df, test_scores_mrmr50_pln_df,
                       header_1="MRMR 50 Emb w/o CL",
                       header_2="MRMR 50 Emb w All")

		posOutcome
			MRMR 50 Emb w/o CL		MRMR 50 Emb w All
		-----------------------------------------------
balanced_accuracy:	72.570%			72.756%

precision_0:		76.271%			78.531%

recall_0:		73.171%			72.585%

precision_1:		68.868%			66.981%

recall_1:		72.277%			73.702%

auc:			77.740%			78.262%



In [28]:
moses83_params = {'subsample': 0.8,
                  'n_estimators': 300,
                  'min_child_weight': 5,
                  'max_depth': 5,
                  'learning_rate': 0.03,
                  'gamma': 2,
                  'colsample_bytree': 0.6}
clf_moses83, cv_moses83, test_scores_moses83 = evaluate_embedding("datasets/embedding-vectors/genexpr_only/property_vector_moses_ft83_2021-01-30.csv", train_df, test_df)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[15:14:09] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:14:09] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 26 minutes and 42.03 seconds.
Best Score: 77.957%
{'subsample': 0.6, 'n_estimators': 300, 'min_child_weight': 4, 'max_depth': 4, 'learning_rate': 0.02, 'gamma': 5, 'colsample_bytree': 0.6}
[15:16:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was ch

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed: 25.9min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.1min finished


ValueError: too many values to unpack (expected 3)

In [27]:
cv_moses83.mean()

NameError: name 'cv_moses83' is not defined

In [57]:
test_scores_moses83.mean()

balanced_accuracy    0.716880
recall_0             0.694737
precision_0          0.781065
recall_1             0.746575
precision_1          0.652695
auc                  0.766609
dtype: float64

In [58]:
#Test score comparison
print_score_comparison(test_scores_mrmr50_df, test_scores_moses83,
                       header_1="MRMR 50", header_2="MOSES 83")


		posOutcome
			MRMR 50		MOSES 83
		-----------------------------------------------
balanced_accuracy:	71.847%			71.688%

precision_0:		76.627%			78.107%

recall_0:		70.190%			69.474%

precision_1:		67.066%			65.269%

recall_1:		73.927%			74.658%

auc:			79.223%			76.661%



In [ ]:

moses83_cl_params, cv_moses83_cl, test_scores_moses83_cl_df = evaluate_embedding("datasets/embedding-vectors/genexpr_clinicaldata/property_vector_moses_ft83-wopln_2021-01-30.csv", pos_outcome_df)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


In [64]:
cv_moses83_cl.mean()

test_balanced_accuracy    0.732873
test_precision_0          0.732185
test_recall_0             0.809721
test_precision_1          0.749274
test_recall_1             0.656025
test_auc                  0.809083
dtype: float64

In [65]:
test_scores_moses83_cl_df.mean()

balanced_accuracy    0.733152
recall_0             0.703325
precision_0          0.813609
recall_1             0.775801
precision_1          0.652695
auc                  0.815124
dtype: float64

In [66]:
print_score_comparison(test_scores_moses83, test_scores_moses83_cl_df,
                       header_1="MOSES 83 w/o CL",
                       header_2="MOSES 83 w CL")

		posOutcome
			MOSES 83 w/o CL		MOSES 83 w CL
		-----------------------------------------------
balanced_accuracy:	71.688%			73.315%

precision_0:		78.107%			81.361%

recall_0:		69.474%			70.332%

precision_1:		65.269%			65.269%

recall_1:		74.658%			77.580%

auc:			76.661%			81.512%



In [69]:
mrmr100_params, cv_mrmr100, test_scores_mrmr100_df = evaluate_embedding("datasets/embedding-vectors/genexpr_only/property_vector_mrmr_ft100_2021-01-29.csv", pos_outcome_df)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[13:56:13] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:56:14] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 25 minutes and 33.28 seconds.
Best Score: 79.597%
{'subsample': 1.0, 'n_estimators': 400, 'min_child_weight': 3, 'max_depth': 5, 'learning_rate': 0.01, 'gamma': 1, 'colsample_bytree': 0.6}
[13:59:32] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only use

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed: 23.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.5min finished


In [70]:
cv_mrmr100.mean()

test_balanced_accuracy    0.721821
test_precision_0          0.736879
test_recall_0             0.758594
test_precision_1          0.710231
test_recall_1             0.685048
test_auc                  0.795970
dtype: float64

In [71]:
test_scores_mrmr100_df.mean()

balanced_accuracy    0.736376
recall_0             0.721763
precision_0          0.775148
recall_1             0.754045
precision_1          0.697605
auc                  0.799344
dtype: float64

In [ ]:
mrmr100_cl_params, cv_mrmr100_cl, test_scores_mrmr100_cl_df = evaluate_embedding("datasets/embedding-vectors/genexpr_clinicaldata/property_vector_mrmr_ft100_wopln_2021-01-29.csv", pos_outcome_df)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


In [73]:
cv_mrmr100_cl.mean()

test_balanced_accuracy    0.714536
test_precision_0          0.721492
test_recall_0             0.779973
test_precision_1          0.719288
test_recall_1             0.649100
test_auc                  0.799172
dtype: float64

In [74]:
test_scores_mrmr100_cl_df.mean()

balanced_accuracy    0.725826
recall_0             0.703704
precision_0          0.786982
recall_1             0.755102
precision_1          0.664671
auc                  0.806984
dtype: float64

In [75]:
print_score_comparison(test_scores_mrmr100_df, test_scores_mrmr100_cl_df,
                       header_1="MRMR 100 Emb w/o CL",
                       header_2="MRMR 100 Emb w CL")

		posOutcome
			MRMR 100 Emb w/o CL		MRMR 100 Emb w CL
		-----------------------------------------------
balanced_accuracy:	73.638%			72.583%

precision_0:		77.515%			78.698%

recall_0:		72.176%			70.370%

precision_1:		69.760%			66.467%

recall_1:		75.405%			75.510%

auc:			79.934%			80.698%



In [17]:
params_mrmr50_nn, clf_mrmr50_nn, cv_scores_mrmr50_nn_df, test_scores_mrmr50_nn_df = evaluate_embedding("datasets/embedding-vectors/genexpr_only/property_vector_mrmr_ft50_notnormalized_2021-02-04.csv", pos_outcome_df, n_jobs=-1)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[12:31:37] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:31:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 26 minutes and 35.45 seconds.
Best Score: 79.358%
{'subsample': 0.8, 'n_estimators': 300, 'min_child_weight': 5, 'max_depth': 5, 'learning_rate': 0.03, 'gamma': 2, 'colsample_bytree': 0.6}
[12:33:36] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only use

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed: 25.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   52.7s finished


In [18]:
cv_scores_mrmr50_nn_df.mean()

0    0.748330
1    0.754550
2    0.797809
3    0.750667
4    0.698851
5    0.793583
dtype: float64

In [19]:
test_scores_mrmr50_nn_df.mean()

balanced_accuracy    0.728803
recall_0             0.706349
precision_0          0.789941
recall_1             0.758503
precision_1          0.667665
auc                  0.779958
dtype: float64

In [31]:
print_score_comparison_cv(cv_scores_mrmr50_df, cv_scores_mrmr50_nn_df, header_1="MRMR 50 Norm", header_2="MRMR 50 Not Norm")

		posOutcome
			MRMR 50 Norm		MRMR 50 Not Norm
		-----------------------------------------------
balanced_accuracy:	74.566%			74.833%

precision_0:		74.920%			75.455%

recall_0:		80.493%			79.781%

precision_1:		75.367%			75.067%

recall_1:		68.639%			69.885%

auc:			79.792%			79.358%



In [22]:
print_score_comparison(test_scores_mrmr50_df, test_scores_mrmr50_nn_df, header_1="MRMR 50 Norm", header_2="MRMR 50 Not Norm")

		posOutcome
			MRMR 50 Norm		MRMR 50 Not Norm
		-----------------------------------------------
balanced_accuracy:	71.847%			72.880%

precision_0:		76.627%			78.994%

recall_0:		70.190%			70.635%

precision_1:		67.066%			66.766%

recall_1:		73.927%			75.850%

auc:			79.223%			77.996%



In [29]:
test_scores_moses83.mean()

NameError: name 'test_scores_moses83' is not defined